In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

In [2]:
print("pandas: ", pd.__version__)
print("seaborn: ",sns.__version__)

pandas:  2.1.0
seaborn:  0.12.2


# Ler microdados do ENEM

## Selecionar apenas uma amostra aleatória de 1% dos alunos
Se a gente lê o arquivo de 3GB inteiro, vai dar problema. Aqui tem uma receita para ler somente x%: https://www.kaggle.com/questions-and-answers/53925 

In [34]:
def skip(i,fraction=0.01):
    if i == 0:
        return False  # precisamos manter o row=0 para ter o header
    else:
        return random.random() > fraction

[skip(i) for i in range(10)]

[False, True, True, True, True, True, True, True, True, True]

A título de exemplo, vamos carregar os dados de 2010.

In [35]:
%time df = pd.read_csv('../inep/enem/2010/DADOS/MICRODADOS_ENEM_2010.csv',skiprows = skip, encoding='latin1',sep=";")

CPU times: user 19.3 s, sys: 916 ms, total: 20.2 s
Wall time: 20.2 s


In [36]:
df

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_ST_CONCLUSAO,TP_ENSINO,CO_MUNICIPIO_ESC,NO_MUNICIPIO_ESC,...,Q48,Q49,Q50,Q51,Q52,Q53,Q54,Q55,Q56,Q57
0,200000000028,2010,7,M,0,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200000000049,2010,2,M,0,3,2,1.0,5107925.0,SORRISO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200000000182,2010,4,M,0,3,1,1.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,G,2.0,3.0,2.0,5.0
3,200000000234,2010,10,M,0,3,3,NaN,NaN,NaN,...,0.0,0.0,0.0,2.0,0.0,E,5.0,5.0,5.0,5.0
4,200000000247,2010,6,F,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46491,200004611354,2010,9,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46492,200004611403,2010,11,M,0,3,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46493,200004611485,2010,14,M,1,3,1,2.0,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,F,1.0,1.0,1.0,1.0
46494,200004611554,2010,16,M,1,2,1,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


De fato, só vem 50 mil linhas (os dados originais devem conter da ordem de 3-6 milhões de participantes)

Em seguinda, vamos filtrar estes dados pela presença na prova, se é treineiro e estar concluindo o EM.

In [215]:
df["TP_PRESENCA_CN"].value_counts()

1    33844
0    12108
2       58
Name: TP_PRESENCA_CN, dtype: int64

In [13]:
df['TP_ST_CONCLUSAO'].value_counts()

TP_ST_CONCLUSAO
1    26733
2    13767
3     5299
Name: count, dtype: int64

## Filtrar os dados por presença, treineiro, EM e notas não-zero

In [39]:
def filterdf(df):
    df = df[df["TP_PRESENCA_CN"] == 1]  # presente nas 4 provas
    df = df[df["TP_PRESENCA_CH"] == 1]
    df = df[df["TP_PRESENCA_LC"] == 1]
    df = df[df["TP_PRESENCA_MT"] == 1]
    if "IN_TREINEIRO" in df.columns:
        df = df[(df["IN_TREINEIRO"] == 0) | (df["IN_TREINEIRO"].isna())]  # não é treineiro ou não existe 
    df = df[df["TP_ST_CONCLUSAO"].isin([1, 2])]  # afirma que concluiu ou vai concluir EM em 2019
    # não queremos as notas == 0
    df = df.query("NU_NOTA_CH != 0 and NU_NOTA_CN != 0 and NU_NOTA_LC != 0 and NU_NOTA_MT != 0")
    df.dropna(subset = ['TX_RESPOSTAS_CN','TX_RESPOSTAS_CH','TX_RESPOSTAS_LC','TX_RESPOSTAS_MT']) # achamos linhas com NaN para estas colunas!
    return df


def load_data(ano,frac):
    'carrega dados com determinado ano e fração'
    print(f"processando {ano} com fraçao = {frac*100:.0f}%",ano, frac)
    def skip(i,fraction=frac):
        if i == 0:
            return False  # precisamos manter o row=0 para ter o header
        else:
            return random.random() > fraction
   
    if ano == 2016:
        caminho = f'../inep/enem/{ano}/DADOS/microdados_enem_{ano}.csv'
    else:
        caminho = f'../inep/enem/{ano}/DADOS/MICRODADOS_ENEM_{ano}.csv'
    #print(caminho)
    df = pd.read_csv(caminho,skiprows = skip, encoding='latin1',sep=";")
    return df
    
def filter_data(df):
    print("filter data...")
    return filterdf(df)

def nome_do_arquivo(ano,frac):
    frac = frac*100
    fn = f'data/enem_{frac:.0f}_{ano}.csv'
    return fn

df = load_data(2014,0.01)
print(len(df))
df = filter_data(df)
print(len(df))


processando 2014 com fraçao = 1% 2014 0.01
86937
filter data...
45264


In [42]:
df['TX_RESPOSTAS_CH'][:10]

0     ADEEECBCCCCAADBABBAEADCEBEDEADCBBEBAAEBECACDA
1     ADBBAABABADBAAEBECACADCECAEEEEECEAADCEBAEEDBC
2     BCADECADBEDDBBCDBACAAEACDEEEBAACDDABCEDECCDDC
5     ADEEECABCADEACBDBECABACEBCEBDAACDEECAEEADEADB
7     EEDDADADECAAEBECACCEAEBACDBABACDECAABCCBBCEDD
8     ECCEDABBDAABBCACECBACABECAAEADACAADCCCAABADDC
10    AABBDDADCACBAACADADABAEEBECCAADDEADAAABEBBACE
12    AABBDDEBDACBCBBAEEDECAEEBECAEABECEBAACCBCBCCA
13    ECEDDDADDCAADBABBCCCACBEABEADCCECDDDAACDDCEEC
14    AACBEAACEACBBAABDEABCDEEBECBEBDECCADCCEDBACCD
Name: TX_RESPOSTAS_CH, dtype: object

In [40]:
df['TX_RESPOSTAS_CH'].describe()

count                                             45264
unique                                            45262
top       CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC
freq                                                  3
Name: TX_RESPOSTAS_CH, dtype: object

## Aqui faremos o trabalho de verdade
Isso deve levar alguns minutos.

In [29]:
frac = 0.01
for ano in range(2014,2023):
    df = load_data(ano,frac)
    df = filter_data(df)
    fn = nome_do_arquivo(ano,frac)
    #print(fn+"...\n")
    df.to_csv(fn,index=False)

import inspect
filter_conditions = inspect.getsource(filterdf)
with open('data/filters.txt','w') as f:
    f.write(filter_conditions)

processando 2014 com fraçao = 1% 2014 0.01
filter data...
processando 2015 com fraçao = 1% 2015 0.01
filter data...
processando 2016 com fraçao = 1% 2016 0.01
filter data...
processando 2017 com fraçao = 1% 2017 0.01
filter data...
processando 2018 com fraçao = 1% 2018 0.01
filter data...
processando 2019 com fraçao = 1% 2019 0.01
filter data...
processando 2020 com fraçao = 1% 2020 0.01
filter data...
processando 2021 com fraçao = 1% 2021 0.01
filter data...
processando 2022 com fraçao = 1% 2022 0.01
filter data...


# Preparar os dados para análise TRI
Precisamos converter as respostas e o gabarito no microdados do INEP para um formato que os pacotes de TRI conseguem ler.

## Funções para extrair acertos
A maioria das análises TRI vão precisar de uma "dicomitização" das respostas. A estrutura de dados que vamos precisar é uma matriz com colunas itens e linhas pessoas com valores 0 e 1 para erros e acertos respectivamente.

Vamos tentar criar umas funções que crie este estrutura de dados.


Temos algumas informações sobre a estrutura dos microdados:
* não vamos usar as primeiras 10 itens da prova LC (Linguágens e Códigos). Desde 2011 (?) estas posições são usadas para as questões de língua estrangeira (espanhol ou inglês). Acredito que somente 40 questões de LC são usados para criar a escala IRT desta prova.
* O INEP divulga, desde o final de 2022, os parámetros IRT (discriminação, dificuldade e c). Além disso, dizem quais itens foram eliminados ("pelo IRT"), porque aparentemente atrapalharam a convergência da estimação do modelo IRT 3PL que estão usando. Vamos ter que tirar estes itens também.  

In [12]:
# Assumimos aqui que as planilhas com informações sobre os itens do INEP já foram convertidos para utf-8 e ficam no diretório DADOS
def load_amostra(ano,perc):
    df = pd.read_csv(f'data/enem_{perc}_{ano}.csv',dtype={'CO_PROVA_LC':int,'CO_PROVA_CN':int,'CO_PROVA_CH':int,'CO_PROVA_MT':int})
    item_info = pd.read_csv(f'../inep/enem/{ano}/DADOS/ITENS_PROVA_{ano}-utf8.csv',sep='\;',engine='python')
    item_info.dropna(subset='CO_ITEM',inplace=True)
    item_info['CO_ITEM'] = item_info['CO_ITEM'].astype(int)
                     
    #print(list(item_info.columns),'\n')
    #print(list(df.columns))
    return df, item_info

df, item_info = load_amostra(2022,1)


In [13]:
df[:2]

,NU_INSCRICAO,NU_ANO,TP_FAIXA_ETARIA,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,TP_NACIONALIDADE,TP_ST_CONCLUSAO,TP_ANO_CONCLUIU,TP_ESCOLA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,210057293916,2022,11,F,1,3,1,1,13,1,...,B,A,A,B,A,A,C,A,A,B
1,210055864562,2022,11,M,1,3,1,1,9,1,...,B,A,A,B,B,A,B,A,A,A


In [14]:
item_info[:2]

,CO_POSICAO,SG_AREA,CO_ITEM,TX_GABARITO,CO_HABILIDADE,IN_ITEM_ABAN,TX_MOTIVO_ABAN,NU_PARAM_A,NU_PARAM_B,NU_PARAM_C,TX_COR,CO_PROVA,TP_LINGUA,IN_ITEM_ADAPTADO
0,46,CH,140506,D,26,0,NaN,0.98121,1.34897,0.06218,AZUL,1055,NaN,0
1,90,CH,111984,A,3,0,NaN,1.90716,1.36087,0.22521,AZUL,1055,NaN,0


In [15]:
def to_acertos(s):
    'resp e gab são strings, retorna uma lista'
    resp = s.iloc[0]
    gab = s.iloc[1]
    return [r == g for r,g in zip(resp,gab)]

def acertos_df(df,exame,as_int=True):
    'Retorna dataframe com acertos. Colunas 1-45 = itens, Linhas = idx do df'
    resp_col = 'TX_RESPOSTAS_' + exame
    gab_col = 'TX_GABARITO_' + exame
    adf = df[[resp_col,gab_col]].apply(to_acertos,axis=1,result_type = "expand")
    adf['acertos'] = adf.sum(axis=1)
    adf['caderno'] = df.loc[:,"CO_PROVA_" + exame]
    if as_int:
        adf = adf.astype("int")
    adf['nota_inep'] = df.loc[:,"NU_NOTA_" + exame]
    adf['candidato'] = df['NU_INSCRICAO']
    adf = adf.set_index('candidato')
    return adf

#item_info = pd.read_csv('data/ITENS_PROVA_2019-utf8.csv')

def reorder_rename_cols(ac,item_info):
    'takes a df with acertos, cuts it up and renames the columns for each caderno and stitches it back again.'
    gb = ac.groupby("caderno")
    
    itemgroups = item_info.groupby("CO_PROVA")
    cadernos = ac['caderno'].value_counts().index[:4] #only the 4 most used cadernos
    groups = []
    for caderno in cadernos:
        #print(caderno)
        itemnames = itemgroups.get_group(caderno).sort_values("CO_POSICAO")["CO_ITEM"].values
        colmap = {i:j for i,j in zip(range(len(itemnames)),itemnames)}
        group = gb.get_group(caderno).rename(colmap,axis=1)
        groups.append(group)
    ac = pd.concat(groups)
    #abondaned_items = item_info.query("IN_ITEM_ABAN == 1")["CO_ITEM"].unique()
    #ac = ac.drop(removed_items,axis=1,errors='ignore')
    abandoned_map = {item:f'{item}-aban' for item in itens.query("IN_ITEM_ABAN == 1").CO_ITEM.unique()}
    ac = ac.rename(columns=abandoned_map)
    return ac

df,itens = load_amostra(2022,1)
ac = acertos_df(df,'MT')
ac = reorder_rename_cols(ac,itens)

In [16]:
ac

,30053,85228,86466,43182,47309,39443-aban,53461,85013,117973,10500,...,82581,31516,68369,60441,117886,95676,19807,acertos,caderno,nota_inep
candidato,,,,,,,,,,,,,,,,,,,,,
210055225226,0,0,0,1,0,0,0,1,0,0,...,0,0,0,1,0,1,0,19,1077,695.8
210055240339,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,10,1077,430.3
210054967269,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,11,1077,499.4
210056223276,0,1,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,8,1077,349.2
210057289803,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,9,1077,525.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210057999183,0,0,0,0,0,0,1,0,0,0,...,0,1,0,1,0,1,0,7,1075,478.2
210057744595,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,7,1075,378.8
210056546986,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,14,1075,655.1


In [59]:
map = {item:f'{item}-aban' for item in itens.query("IN_ITEM_ABAN == 1").CO_ITEM.unique()}
map

{84199: '84199-aban',
 39443: '39443-aban',
 89637: '89637-aban',
 83727: '83727-aban',
 75845: '75845-aban',
 64203: '64203-aban',
 141605: '141605-aban',
 89554: '89554-aban'}

In [17]:
perc = 1
for ano in range(2009,2023):
    df,itens = load_amostra(ano,perc)
    for area in ['CH', 'CN', 'MT']: # LC tem o problema de 50 itens no gabarito
        ac = acertos_df(df,area)
        ac = reorder_rename_cols(ac,itens)
        ac.to_csv(f'data/ac_{perc}_{ano}_{area}.csv',index=True)

